<a href="https://colab.research.google.com/github/k1151msarandega/Lapicque-s-RC/blob/main/Lapicque's_RC_rate_encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Architecture:** *Lapicque's RC*


**Encoding Scheme:** *Rate encoding*


1. Import the required libraries and modules:

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import snntorch as snn


2. Define the network architecture:

In [ ]:
class SNNModel(nn.Module):
    def __init__(self):
        super(SNNModel, self).__init__()
        self.fc1 = snn.Conv2d(1, 32, kernel_size=3)
        self.fc2 = snn.Conv2d(32, 64, kernel_size=3)
        self.fc3 = snn.Linear(256, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x


3. Load the MNIST dataset:

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)


4. Create an instance of the SNN model and define the loss function and optimizer:

In [ ]:
model = SNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


5. Train the SNN model:

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(10):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        spiking_inputs = snn.io.rate(inputs, duration=100, dt=1.0)
        outputs = model(spiking_inputs)

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).sum().item()

        running_loss += loss.item()

    accuracy = 100.0 * correct / total
    print(f"Epoch: {epoch+1}, Loss: {running_loss/len(train_loader)}, Accuracy: {accuracy}%")


6. Test the SNN model:

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)

        spiking_inputs = snn.io.rate(inputs, duration=100, dt=1.0)
        outputs = model(spiking_inputs)

        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).sum().item()

accuracy = 100.0 * correct / total
print(f"Test Accuracy: {accuracy}%")
